In [34]:
import random
import pandas as pd
import numpy as np

# Data

In [35]:
df = pd.read_csv('Titanic_dataset.csv')
df.head()

df.loc[df['Age'].isnull(),'Age'] = np.round(df['Age'].mean())
df.loc[df['Embarked'].isnull(),'Embarked'] = df['Embarked'].value_counts().index[0]

features = ['Pclass','Sex','Age','SibSp','Parch', 'Fare', 'Embarked']
print(df[features].head(10))




     Pclass     Sex   Age  SibSp  Parch     Fare Embarked
16        3    male   2.0      4      1  29.1250        Q
472       2  female  33.0      1      2  27.7500        S
774       2  female  54.0      1      3  23.0000        S
678       3  female  43.0      1      6  46.9000        S
873       3    male  47.0      0      0   9.0000        S
386       3    male   1.0      5      2  46.9000        S
316       2  female  24.0      1      0  26.0000        S
237       2  female   8.0      0      2  26.2500        S
320       3    male  22.0      0      0   7.2500        S
643       3    male  30.0      0      0  56.4958        S


# Define model

In [36]:
import math

def renyi_entropy(ps):
    e = 0
    p = 0
    
    #Для alpha=1 вместо энтропии Реньи считаем энтропию Шеннона
    if alpha == 1:
        if len(ps) > 0:
            for i in range(n_classes):
                p = (ps.count(i)/len(ps))
                if p == 0:
                    e = e - 0
                else:
                    e = e - p * math.log2(p)
        else:
            e = 0
        
    #Для остальных значений alpha - считаем энтропию Реньи. Внимание: по определению alpha>0 должно быть
    else:
        if len(ps) > 0:
            for i in range(n_classes):
                p += (ps.count(i)/len(ps))**alpha 
                
            #здесь используется логарифм по основанию 2        
            e = (1/(1-alpha))*math.log2(p)
            # e = 1/(1-alpha)*math.log(p)
        else:
            e = 0
        
        
    #print("Alpha: ", alpha) 
    return e
    
    
def information_gain(left_child, right_child):
    parent = left_child + right_child
    IG_p = renyi_entropy(parent)
    IG_l = renyi_entropy(left_child)
    IG_r = renyi_entropy(right_child)
    
    return IG_p - (len(left_child) / len(parent)) * IG_l - (len(right_child) / len(parent)) * IG_r

In [37]:
def draw_bootstrap(X_train, y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
    X_bootstrap = X_train.iloc[bootstrap_indices].values
    y_bootstrap = y_train[bootstrap_indices]
    X_oob = X_train.iloc[oob_indices].values
    y_oob = y_train[oob_indices]
    return X_bootstrap, y_bootstrap, X_oob, y_oob

def oob_score(tree, X_test, y_test):
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != y_test[i]:
            mis_label += 1
    return mis_label / len(X_test)

In [38]:
def find_split_point(X_bootstrap, y_bootstrap, max_features):
    feature_ls = list()
    num_features = len(X_bootstrap[0])

    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)

    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'y_bootstrap': []}

        # split children for continuous variables
            if type(split_point) in [int, float]:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value <= split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
        # split children for categoric variables
            else:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value == split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])

            split_info_gain = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                node = {'information_gain': split_info_gain,
                        'left_child': left_child,
                        'right_child': right_child,
                        'split_point': split_point,
                        'feature_idx': feature_idx}


    return node

In [39]:
def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key = y_bootstrap.count)
    return pred


def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

In [40]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    return root_node

def random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_samples_split):
    tree_ls = list()
    oob_ls = list()
    for i in range(n_estimators):
        X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)
        tree = build_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split)
        tree_ls.append(tree)
        oob_error = oob_score(tree, X_oob, y_oob)
        oob_ls.append(oob_error)
    print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
    print("Alpha: ", alpha)
    return tree_ls

In [41]:
def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']
    
    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']

def predict_rf(tree_ls, X_test):
    pred_ls = list()
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test.values[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
    return np.array(pred_ls)

# Test with cross validation

In [ ]:
#использование кросс-валидации
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

#загружаем данные
df = pd.read_csv('Titanic_data.csv')
df.head()

df.loc[df['Age'].isnull(),'Age'] = np.round(df['Age'].mean())
df.loc[df['Embarked'].isnull(),'Embarked'] = df['Embarked'].value_counts().index[0]

features = ['Pclass','Sex','Age','SibSp','Parch', 'Fare', 'Embarked']

X = df[features]
y = df['Survived']


#задаем параметры модели случайного леса  
n_classes=2 #число классов в нашей задаче классификации
n_estimators=10 #число деревьев
max_features=3  #число фич, которые рассматриваются при расщеплении
max_depth=10 # глубина деревьев
min_samples_split=2 #минимальное количество наблюдений в узле

#создаем файл для записи результатов
myfile = 'Random_forest_Titanic_Renyi.csv'

with open(myfile, 'a') as file: 
    file.write('alpha' + ';' + 'n_estimators' + ';' +  'max_features' + ';' + 'max_depth' + ';' + 'accuracy' + ';' + 'f1_score' + ';' + '\n')
    
results=[]

#разбиваем датасет на части для кросс-валидации и затем на этих кусочках запускаем моедль
kf = KFold(n_splits=10, random_state=0, shuffle=True) # Define the split - into 10 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf) 
KFold(n_splits=10, random_state=0, shuffle=True)

#цикл по разбиениям датасета
for train_index, test_index in kf.split(X):
    X_train = df.iloc[train_index].loc[:, features]
    X_test = df.iloc[test_index][features]
    y_train = df.iloc[train_index].loc[:,'Survived'].values
    y_test = df.loc[test_index]['Survived'].values
    
    
    #цикл по значениям параметра alpha
    for i in np.arange(0.01,1.01,0.01):
        # задаем параметр деформации ----alpha=1 означает обычную энтропию Шэннона
        alpha = i
        # задаем модель
        model = random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_samples_split)
        # делаем предсказание
        preds = predict_rf(model, X_test)
    
        acc = sum(preds == y_test) / len(y_test)
        f = f1_score(y_test, preds, labels=np.unique(preds))
        results.append((alpha, acc, f))
        
        
    
        #записываем результаты в файл
        with open(myfile, 'a') as file: 
            file.write(str(alpha) + ';' + str(n_estimators) + ';' + str(max_features) + ';' 
                             + str(max_depth) + ';' + str(acc) + ';' + str(f) + ';' + '\n')

results = pd.DataFrame(results, columns=['alpha', 'acc', 'f1'])

grouped = results.groupby('alpha')
print(grouped.aggregate(np.mean))
grouped.aggregate(np.mean).to_csv("Cross_validation_results_Titanic.csv")

KFold(n_splits=10, random_state=0, shuffle=True)
OOB estimate: 0.41
Alpha:  0.01
OOB estimate: 0.40
Alpha:  0.02


C:\Users\verai\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


OOB estimate: 0.38
Alpha:  0.03
OOB estimate: 0.36
Alpha:  0.04
OOB estimate: 0.39
Alpha:  0.05
OOB estimate: 0.39
Alpha:  0.060000000000000005
OOB estimate: 0.36
Alpha:  0.06999999999999999
OOB estimate: 0.36
Alpha:  0.08
OOB estimate: 0.31
Alpha:  0.09
OOB estimate: 0.31
Alpha:  0.09999999999999999
OOB estimate: 0.33
Alpha:  0.11
OOB estimate: 0.32
Alpha:  0.12
OOB estimate: 0.30
Alpha:  0.13
OOB estimate: 0.34
Alpha:  0.14
OOB estimate: 0.35
Alpha:  0.15000000000000002
OOB estimate: 0.31
Alpha:  0.16
OOB estimate: 0.37
Alpha:  0.17
OOB estimate: 0.31
Alpha:  0.18000000000000002
OOB estimate: 0.31
Alpha:  0.19
OOB estimate: 0.34
Alpha:  0.2
OOB estimate: 0.33
Alpha:  0.21000000000000002
OOB estimate: 0.31
Alpha:  0.22
OOB estimate: 0.29
Alpha:  0.23
OOB estimate: 0.32
Alpha:  0.24000000000000002
OOB estimate: 0.33
Alpha:  0.25
OOB estimate: 0.36
Alpha:  0.26
OOB estimate: 0.32
Alpha:  0.27
OOB estimate: 0.36
Alpha:  0.28
OOB estimate: 0.32
Alpha:  0.29000000000000004
OOB estimate: 0.

OOB estimate: 0.30
Alpha:  0.33
OOB estimate: 0.31
Alpha:  0.34
OOB estimate: 0.32
Alpha:  0.35000000000000003
OOB estimate: 0.34
Alpha:  0.36000000000000004
OOB estimate: 0.34
Alpha:  0.37
OOB estimate: 0.28
Alpha:  0.38
OOB estimate: 0.33
Alpha:  0.39
OOB estimate: 0.31
Alpha:  0.4
OOB estimate: 0.33
Alpha:  0.41000000000000003
OOB estimate: 0.31
Alpha:  0.42000000000000004
OOB estimate: 0.27
Alpha:  0.43
OOB estimate: 0.36
Alpha:  0.44
OOB estimate: 0.36
Alpha:  0.45
OOB estimate: 0.35
Alpha:  0.46
OOB estimate: 0.34
Alpha:  0.47000000000000003
OOB estimate: 0.30
Alpha:  0.48000000000000004
OOB estimate: 0.31
Alpha:  0.49
OOB estimate: 0.34
Alpha:  0.5
OOB estimate: 0.28
Alpha:  0.51
OOB estimate: 0.29
Alpha:  0.52
OOB estimate: 0.30
Alpha:  0.53
OOB estimate: 0.31
Alpha:  0.54
OOB estimate: 0.30
Alpha:  0.55
OOB estimate: 0.33
Alpha:  0.56
OOB estimate: 0.30
Alpha:  0.5700000000000001
OOB estimate: 0.35
Alpha:  0.5800000000000001
OOB estimate: 0.28
Alpha:  0.59
OOB estimate: 0.35
A